In [1]:
import numpy as np

In [2]:
m=10000
n=100
r=20
#epsilon iid suivant une N(0,1) (sigma = 1 => beta = 1)
beta = 1
# On fixe a et b arbitrairement comme dit sur le paper :
prior_gamma_params={"alpha":1, "beta": 1}

In [75]:
A_gen = np.random.normal(loc=0,scale=1,size=(m, r))
B_gen = np.random.normal(loc=0,scale=1,size=(n, r))
X_gen_origin = A_gen.dot(B_gen.T)
X_gen = X_gen_origin + np.random.normal(loc=0,scale=0.01,size=(m,n))

In [76]:
def get_gamma_param(A, B, a, b, A_mean, B_mean, Sigma_a, Sigma_b, col_num):
    m=A.shape[0]
    n=B.shape[0]
    m_a= A_mean[:,col_num].T.dot(A_mean[:,col_num]) + m* Sigma_a[col_num,col_num]
    m_b= B_mean[:,col_num].T.dot(B_mean[:,col_num]) + m* Sigma_b[col_num,col_num]
    alpha = a+(m+n)/2.*r
    beta = (2*b + m_a + m_b)/2
    return alpha, beta

In [77]:
#initialiser A et B
A = np.random.normal(loc=0,scale=1,size=(m, r))
B = np.random.normal(loc=0,scale=1,size=(n, r))
Gamma = np.diag(np.array([1./x for x in np.random.gamma(prior_gamma_params["alpha"], prior_gamma_params["beta"], r)]))
#initialiser les moyennes par les valeurs initiales
B_mean = B 
A_mean = A
#initialiser Sigma_b comme l'identité :
Sigma_b = np.identity(r)


In [78]:
import scipy
max_step = 100
for step in range(max_step):
    print "step",step,":", np.linalg.norm(A_mean.dot(B_mean.T)-X_gen_origin)
    print np.linalg.norm(A_mean.dot(B_mean.T)-X_gen)
    for i in range(m):
        #calcul de la variance
        Sigma_a = prior_gamma_params["beta"]*np.linalg.inv((B_mean.T).dot(B_mean)+ Gamma)
        mean_a_line = prior_gamma_params["beta"]* Sigma_a.dot(B_mean.T).dot(X_gen[i,:].T)
        A[i,:] = np.random.multivariate_normal(mean_a_line,Sigma_a)
        #Mise à jour de la moyenne
        A_mean[i] = mean_a_line
    
    for j in range(n):
        Sigma_b = prior_gamma_params["beta"]*np.linalg.inv((A_mean.T).dot(A_mean)+ Gamma)
        mean_b_line = prior_gamma_params["beta"]* Sigma_b.dot(A_mean.T).dot(X_gen[:,j].T)
        B[j,:] = np.random.multivariate_normal(mean_b_line,Sigma_b)
        #Mise à jour de la moyenne
        B_mean[j] = mean_b_line
    for k in range(r):
        alpha, beta= get_gamma_param(
            A, B, prior_gamma_params["alpha"], prior_gamma_params["beta"],A_mean,B_mean, Sigma_a, Sigma_b, k)
        Gamma[k,k] = 1./np.random.gamma(alpha, 1./beta)    

print "step",step+1,":", np.linalg.norm(A_mean.dot(B_mean.T)-X_gen_origin)
    

step 0 : 6381.62130834
6381.63087878
step 1 : 861.703264421
861.742954915
step 2 : 6.0592822585
9.82888608754
step 3 : 5.94778376562
9.76018219143
step 4 : 5.92536762754
9.74647350264
step 5 : 5.90189004861
9.73214419065
step 6 : 5.8741982329
9.71529393144
step 7 : 5.85827269655
9.70562214903
step 8 : 5.82930235861
9.6880789935
step 9 : 5.80780953149
9.67509665758
step 10 : 5.78437146541
9.66096339985
step 11 : 5.76778107573
9.65098635352
step 12 : 5.75003716797
9.64034584612
step 13 : 5.72968208062
9.62816045573
step 14 : 5.70969580497
9.61622107333
step 15 : 5.68945394616
9.60414325822
step 16 : 5.67424310174
9.59509090991
step 17 : 5.65613970931
9.58434141579
step 18 : 5.63989587906
9.57470601959
step 19 : 5.617997621
9.56174864396
step 20 : 5.60349815139
9.5531978183
step 21 : 5.59214945467
9.54651077012
step 22 : 5.57188112353
9.53457853039
step 23 : 5.560031319
9.52761398652
step 24 : 5.54026725568
9.51604234754
step 25 : 5.5289372217
9.50940673972
step 26 : 5.51657532461
9.50218

In [ ]:
np.linalg.norm(A.dot(B.T)-X_gen)

In [ ]:
mean_a_line.shape, Sigma_a.shape

In [42]:
np.min(Sigma_a)

0.0

In [26]:
mean_a_line[i] = V

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [4]:
X=np.zeros((m,n))
cols_to_fill=np.random.choice(n,r,replace=False)
for k in cols_to_fill:
    X[:,k]=np.random.normal(0,1,m)
for k in (set(range(n))-set(cols_to_fill)):
    rand_coefs = np.random.normal(0,1,r)
    X[:,k]=sum([X[:,cols_to_fill[i]]*rand_coefs[i] for i in range(r)])

In [5]:
mean_a

array([[  0.92284887,  -2.47007301,  -2.10590224, ...,  -1.58812097,
          3.05882318,  -0.30392471],
       [  5.12940119,  -0.44404491,  -3.0105268 , ...,   0.23829583,
          1.24466971,  -2.93559611],
       [ -2.97957144,   0.53661026,  -1.47360067, ...,   1.66889876,
          0.13199076,  -2.01840337],
       ..., 
       [ -6.65860565,   2.94412603,  -0.92336414, ...,  -1.41649424,
         -2.74000717,   1.95974028],
       [  3.89330264,   0.84279169,   1.09092452, ...,  -0.72211154,
          5.96339864,  -2.33928196],
       [  6.2822498 ,   2.42412484,   4.94979749, ...,  -0.3605845 ,
         -2.21448133,  10.24788361]])

In [6]:
p=0.3
n_known_coefs=int(p*m*n)
pick_coefs=np.random.choice(m*n, n_known_coefs, replace=False)
known_coefs = [(i%m, i/m) for i in pick_coefs]

In [7]:
#Initialisation
#Les matrices et paramètres
A=np.zeros((m,r))
B=np.zeros((n,r))
gamma=np.random.normal(0,1,r)
beta=.1

In [8]:
#Initialisation de Sigma
Sigma_A = [np.linalg.inv(np.diag(gamma[i])) for i in range(r)]
for i in range(m):
    mean = 
    A[i] = np.random.multivariate_normal

SyntaxError: invalid syntax (<ipython-input-8-e1a37ca03106>, line 4)